## **GUVI_LLM_GPT_MODEL...**

In [1]:
!pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip list | grep torch

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip uninstall torch torchvision torchaudio


^C


In [3]:
!pip install torch torchvision torchaudio

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.3 MB 5.1 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.3 MB 3.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.3 MB 3.1 MB/s eta 0:00:01
   ----------------- ---------------------- 0.6/1.3 MB 3.0 MB/s eta 0:00:01
   ---------------------- ----------------- 0.7/1.3 MB 3.3 MB/s eta 0:00:01
   -------------------------- ------------- 0.8/1.3 MB 3.4 MB/s eta 0:00:01
   -------------------------- ------------- 0.8/1.3 MB 3.4 MB/s eta 0:00:01
   -------------------------- ------------- 0.8/1.3 MB 3.4 MB/s eta 0:00:01
   ---------------------------------- ----- 1.1/1.3 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 2.8 MB/s eta 0:00:01
   ----------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------  1.3/1.3 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 997.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 6.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.4 MB 6.1 MB/s eta 0:00:01
   -------------- ------------------------- 0.9/2.4 MB 5.1 MB/s eta 0:00:01
   ------------------ --------------------- 1.1/2.4 MB 5.0 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.4 MB 4.6 MB/s eta 0:00:01
   ------------------------ --------------- 1.5/2.4 MB 4.5 MB/s eta 0:00:01
   --------------------------- ------------ 1.6/2.4 MB 4.3 MB/s eta 0:00:01
   ------------------------------ --------- 1.9/2.4 MB 4.4 MB/s eta 0:00:01
   --------------------

In [2]:
!pip install transformers datasets torch fastapi uvicorn


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive

ModuleNotFoundError: No module named 'google.colab'

In [5]:
import os
import re
from transformers import GPT2Tokenizer

def preprocess_data(input_file, output_file, tokenizer_name="gpt2"):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_name)
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    with open(output_file, 'w', encoding='utf-8') as f:
        for line in lines:
            line = line.lower()

            # Tokenize the line
            token_ids = tokenizer.encode(line, add_special_tokens=False)
            # Convert token IDs back to tokens
            tokenized_line = tokenizer.convert_ids_to_tokens(token_ids)
            # Convert tokens to text and remove special tokens
            processed_line = " ".join(tokenized_line).replace('Ġ', '').replace('Ċ', '').replace('�', '').strip()

            processed_line = re.sub(r'[^a-zA-Z0-9\s,():%&]', '', processed_line)
            processed_line = re.sub(r'\bgu vi\b', 'guvi', processed_line)

            # Remove extra spaces
            processed_line = re.sub(r'\s+', ' ', processed_line)
            # Write the processed line to the output file
            f.write(processed_line + "\n")

input_file = r"C:\Users\TOSHIBA\Downloads\guvi_data.txt"  # Make sure this path is correct
output_file = "processed_GUVI_data.txt"
preprocess_data(input_file, output_file)

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\TOSHIBA\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Create dataset
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )

train_dataset = load_dataset(output_file, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Step,Training Loss


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json')

# **Model Testing**

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the fine-tuned model and tokenizer
model_name_or_path = "./fine_tuned_model"
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)

# Set the padding token to the EOS token
tokenizer.pad_token = tokenizer.eos_token

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the text generation function
def generate_text(model, tokenizer, seed_text, max_length=100, temperature=1.0, num_return_sequences=1):
    # Tokenize the input text
    inputs = tokenizer.encode_plus(seed_text, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Generate text
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
            attention_mask=attention_mask
        )

    # Decode the generated text
    generated_texts = []
    for i in range(num_return_sequences):
        generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
        generated_texts.append(generated_text)

    return generated_texts

In [ ]:
# Test the model
seed_text = input()
generated_texts = generate_text(model, tokenizer, seed_text, max_length=50, temperature=0.0001, num_return_sequences=1)

for i, text in enumerate(generated_texts):
    print(f"Generated Text : \n{text}\n")


GUVI Machine Learning Course
Generated Text : 
GUVI Machine Learning Course

guvi offers a comprehensive course on machine learning, including machine learning, machine learning algorithms, machine learning frameworks, and more, covering topics such as machine learning, machine learning algorithms, machine learning frameworks, and more,



In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install mysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.2.4-cp310-cp310-linux_x86_64.whl size=124738 sha256=383a0316ebc49acddc02c86204a087cb50c00f0b5584025d7a6b589d5e1318a6
  Stored in directory: /root/.cache/pip/wheels/ac/96/ac/2a4d8cb58a4d95de1dffc3f8b0ea42e0e5b63ab97640edbda3
Successfully built mysqlclient


In [ ]:
import streamlit as st
import mysql.connector
import bcrypt
import datetime
import re
import json
import torch
import pytz
from transformers import GPT2LMHeadModel, GPT2Tokenizer

ModuleNotFoundError: No module named 'mysql'